In [ ]:
import numpy as np # linear algebra
import math
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import cv2
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax

In [ ]:
train_dir = "../input/emotion-detection-fer/train" #train images directory
test_dir = "../input/emotion-detection-fer/test"   #test images directory
siz = 96
batch_siz = 128

In [ ]:
train_datagen = ImageDataGenerator(#rotation_range = 10,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   horizontal_flip = True,
                                   rescale = 1./255,
                                   #zoom_range = 0.2,
                                   validation_split = 0.2
                                  )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                        validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (siz, siz),
                                                    batch_size = batch_siz,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )
validation_generator = validation_datagen.flow_from_directory( directory = test_dir,
                                                              target_size = (siz, siz),
                                                              batch_size = batch_siz,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation"
                                                             )

In [ ]:
model= tf.keras.models.Sequential(
    [
        Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(siz, siz,1)),
        Conv2D(64,(3,3), padding='same', activation='relu' ),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(128,(5,5), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Flatten(),
        
        Dense(512,activation = 'relu'),
        BatchNormalization(),
        Dropout(0.15),
        
        Dense(256, activation = 'relu'),
        Dropout(0.15),
        
        Dense(128, activation = 'relu'),
        Dropout(0.15),
        
        Dense(64, activation = 'relu'),
        Dropout(0.15),
        
        Dense(32, activation = 'relu'),
        Dropout(0.15),
        
        Dense(7, activation='softmax'),
    ]
)
model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
history = model.fit(x = train_generator,epochs = 60, validation_data = validation_generator)

In [ ]:
#Mathematical model

# PREVIOUS
'''
emotion = 2 #GET THE EMOTION FROM THE PICTURE, FOR EXAMPLE HAPPY: 2
timeSpentOnPicture = 60 #GET THE TIME SPENT ON THE PICTURE FROM THE APP, FOR EXAMPLE: 60
score = emotion * math.log10(timeSpentOnPicture)
skipped = False #GET WETHER THE USER SKIPPED THE PICTURE OR NOT 
if skipped == True:
    score -= emotion * math.log10(timeSpentOnPicture)
else:
    score += emotion * math.log10(timeSpentOnPicture)
'''

# IMPROVED

emotion = 2 #GET THE EMOTION FROM THE PICTURE, FOR EXAMPLE HAPPY: 2
timeSpentOnPicture = 60 #GET THE TIME SPENT ON THE PICTURE FROM THE APP, FOR EXAMPLE: 60
score = (emotion * math.log10(timeSpentOnPicture)) + 1
skipped = False #GET WHETHER THE USER SKIPPED THE PICTURE OR NOT 
if skipped == True:
    score += math.sqrt(score)
else:
    score = math.sqrt(score)